In [2]:
import signal
import threading
from collections import defaultdict
from typing import Dict

from ipyleaflet import Circle, LayerGroup, Map
from palettable.colorbrewer.qualitative import Set1_9, Set3_6
from visionapi.messages_pb2 import SaeMessage
from visionlib.pipeline.consumer import RedisConsumer

# Setup graceful exit on SIGINT
stop_event = threading.Event()

def sig_handler(signum, _):
    signame = signal.Signals(signum).name
    print(f'Caught signal {signame} ({signum}). Exiting...')
    stop_event.set()

signal.signal(signal.SIGTERM, sig_handler)
signal.signal(signal.SIGINT, sig_handler)


CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set3_6.hex_colors

# Map setup
INIT_CENTER = (39.96871094145749, -86.12691605973491)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=14)
marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)

displayed_objects: Dict[str, Dict[str, Circle]] = defaultdict(lambda: defaultdict(dict))

# Start listening to SAE messages
consume = RedisConsumer('localhost', 6379, ['geomapper:stream1'], block=200)
# consume = RedisConsumer('localhost', 6379, ['geomerger:merged'], block=200)

with consume:
    for stream_key, proto_data in consume():

        if stop_event.is_set():
            break

        if stream_key is None:
            continue

        sae_msg = SaeMessage()
        sae_msg.ParseFromString(proto_data)

        # Update map center with the first detected object (should only happen once)
        if DETECTED_CENTER is None and len(sae_msg.detections) > 0:
            DETECTED_CENTER = (sae_msg.detections[0].geo_coordinate.latitude, sae_msg.detections[0].geo_coordinate.longitude)
            m.center = DETECTED_CENTER
            m.zoom = 19

        # Remove objects from cache and map that were rendered before but do not exist anymore
        removed_objects = list(filter(lambda id: id not in map(lambda d: d.object_id, sae_msg.detections), displayed_objects[stream_key].keys()))
        for id in removed_objects:
            removed = displayed_objects[stream_key].pop(id, None)
            if removed is not None:
                marker_layer.remove(removed)

        # Add new objects / Update existing objects locations
        for detection in sae_msg.detections:
            lat, lon = detection.geo_coordinate.latitude, detection.geo_coordinate.longitude
            if detection.object_id in displayed_objects[stream_key]:
                displayed_objects[stream_key][detection.object_id].location = (lat, lon)
            else:
                marker = Circle(location=[lat, lon], radius=2, color=STREAM_CMAP[hash(stream_key) % len(STREAM_CMAP)], fill_color=CLS_CMAP[detection.class_id], fill_opacity=0.8)
                displayed_objects[stream_key][detection.object_id] = marker
                marker_layer.add(marker)

Map(center=[39.96871094145749, -86.12691605973491], controls=(ZoomControl(options=['position', 'zoom_in_text',…

ConnectionError: Connection closed by server.

In [1]:
import cameratransform as ct
import numpy as np
from ipyleaflet import Circle, LayerGroup, Map
from palettable.colorbrewer.qualitative import Set1_9, Set3_6
import matplotlib.pyplot as plt
import cv2

im = plt.imread("HazelDell126thSt.jpg") #roundaboutElmStreetRangeLineRd.jpg 
(im_height_px,im_width_px, _)=im.shape

#define the camnera 

camera = ct.Camera(projection=ct.RectilinearProjection(image=im,view_x_deg=100),
                   orientation=ct.SpatialOrientation(heading_deg=325,roll_deg=0),
                   lens=ct.BrownLensDistortion(k1=None,k2=None,k3=None))

camera.setGPSpos(39.97095, -86.070464) # (lat,lon)

lm_points_px = np.array([[1910,556],[918,454],[3578,1028],[2670,426],[388,1472],[1940,297],[291,588],[2388,1052],[1138,1114],[678,1480],[3058,1154],[2214,1876],[1172,1842],[484,285],[2961,281],[3216,1705]])
lm_points_gps = np.array([
    (39.97120131678513, -86.07069301987889, 0),
    (39.97117973342699, -86.07099074506144, 0),
    (39.97122649736122, -86.07040401186067, 0),
    (39.971431024964446, -86.07064876341829, 0),
    (39.970973692516665, -86.07065330551558, 0),
    (39.97144955403607, -86.07091817364868, 0),
    (39.971055885929694, -86.07105511808322, 0),
    (39.97112117910931, -86.0705567460026, 0),
    (39.97104374840547, -86.0706531424601, 0),
    (39.970990817654894, -86.07063503755037, 0),
    (39.97114498477538, -86.0704808105414, 0),
    (39.971025762232664, -86.07051500870425,0),
    (39.970994138962425, -86.07056942252174,0),
    (39.97125443083863, -86.07159711359199,0),
    (39.97193973100817, -86.07068673844802,0),
    (39.97107486291342, -86.07044793785327,0)
])# lm_points_gps_string = np.array()
# for index,gps in enumerate(lm_points_gps):
#      np.append(lm_points_gps_string,camera.setGPSpos(gps),axis=0)



lm_points_space = camera.spaceFromGPS(lm_points_gps) # convert the GPS to local space coordinate format
print(lm_points_space)
# 39.97126541853717, -86.07065908281929

# camera.SpatialOrientation(heading_deg=325)
camera.addLandmarkInformation(lm_points_px, lm_points_space, [2,2,3])


trace = camera.metropolis([
        ct.FitParameter("elevation_m", lower=0, upper=20, value=5),
        ct.FitParameter("tilt_deg", lower=45, upper=120, value=85),
        ct.FitParameter("k1",lower=-1.5,upper=1.5,value=0.1),
        ct.FitParameter("k2",lower=-0.1,upper=0.1,value=0),
        ], iterations=2e3)


# print("All Camera Parameters After Fitting:")
# for attr, value in camera.__dict__.items():
#     print(f"{attr}: {value}")

# camera.plotTrace()
# plt.tight_layout()
# plt.show()

# camera.plotFitInformation(im)
# plt.legend()
# plt.savefig('fitted.jpg')

# topview_im = camera.getTopViewOfImage(
#         im,
#         extent=(-100,20,-20,100),
#         scaling=0.06
#     )
# print(type(topview_im))
# cv2.imwrite('topview.jpg',topview_im)

# # # plt.savefig(    )
# # Plot the image with landmarks
# plt.figure(figsize=(10, 8))
# plt.imshow(im)
# plt.scatter(lm_points_px[:, 0], lm_points_px[:, 1], color='red', marker='o', s=20, label='Landmarks')

# plt.title("Landmark Points on Image")
# plt.xlabel("X Pixels")
# plt.ylabel("Y Pixels")
# plt.legend()

# plt.savefig('landmark.jpg')


###############################FUNCTION FOR LATER USE###########################
# Map setup

CLS_CMAP = Set1_9.hex_colors
STREAM_CMAP = Set3_6.hex_colors

INIT_CENTER = (39.971201044200726, -86.07069033678215)
DETECTED_CENTER = None

m = Map(center=INIT_CENTER, zoom=35)

marker_layer = LayerGroup()
m.add(marker_layer)
m.layout.height = '800px'
display(m)
lat,lon = 39.97109148165551, -86.07059324551214
marker = Circle(location=[lat, lon], radius=2, color=STREAM_CMAP[0], fill_color=CLS_CMAP[2], fill_opacity=0.8)
marker_layer.add(marker)

[[-19.51619702  27.94517654   0.        ]
 [-44.88721107  25.54532411   0.        ]
 [  5.11195762  30.74510552   0.        ]
 [-15.74477802  53.48755194   0.        ]
 [-16.13194479   2.63450477   0.        ]
 [-38.70279846  55.54797294   0.        ]
 [-50.37292646  11.77414511   0.        ]
 [ -7.90346842  19.03425261   0.        ]
 [-16.11803326  10.42436416   0.        ]
 [-14.57520992   4.53873012   0.        ]
 [ -1.43253114  21.68131793   0.        ]
 [ -4.3467776    8.42437715   0.        ]
 [ -8.9837308    4.908034     0.        ]
 [-96.55952587  33.85177814   0.        ]
 [-18.98071248 110.05309055   0.        ]
 [  1.36875716  13.88412262   0.        ]]


100%|██████████| 2000/2000 [00:02<00:00, 709.35it/s, acc_rate=0.00875, factor=0.9]

      elevation_m   tilt_deg        k1        k2    probability
0        6.424329  66.206765  0.477254 -0.085717 -121113.161787
1        6.424329  66.206765  0.477254 -0.085717 -121113.161787
2        6.424329  66.206765  0.477254 -0.085717 -121113.161787
3        6.424329  66.206765  0.477254 -0.085717 -121113.161787
4        6.424329  66.206765  0.477254 -0.085717 -121113.161787
...           ...        ...       ...       ...            ...
1794    10.295603  60.545271  0.407833 -0.053957 -114617.232727
1795    10.295603  60.545271  0.407833 -0.053957 -114617.232727
1796    10.295603  60.545271  0.407833 -0.053957 -114617.232727
1797    10.295603  60.545271  0.407833 -0.053957 -114617.232727
1798    10.295603  60.545271  0.407833 -0.053957 -114617.232727

[1799 rows x 5 columns]
0      -121113.161787
1      -121113.161787
2      -121113.161787
3      -121113.161787
4      -121113.161787
            ...      
1794   -114617.232727
1795   -114617.232727
1796   -114617.232727
1797   -1

Map(center=[39.971201044200726, -86.07069033678215], controls=(ZoomControl(options=['position', 'zoom_in_text'…